# Import statements

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import string
import re
from pandas import DataFrame, Series
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from gensim.models.tfidfmodel import TfidfModel
from gensim.corpora.dictionary import Dictionary
from wordcloud import WordCloud

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Function to rename long-worded columns

In [0]:
def rename_col(csv, og_name, new_name):
    csv.rename(columns={og_name:new_name},inplace=True)

# Import csv files as dataframes

In [0]:
stud = pd.read_csv('Difficulty Protocol Data - Student Protocol.csv',encoding="utf-8",index_col=0)
profta = pd.read_csv('Difficulty Protocol Data - Faculty_TA Protocol.csv',encoding="utf-8",index_col=0)

# Rename columns

In [0]:
rename_col(stud,'Timestamp','time')
rename_col(stud,'Participant ID','id')
rename_col(stud, 'What topics did you cover in class this week?','topics_covered')
rename_col(stud,'What kinds of activities did you focus on out of class this week?  [Reading/Research]','reading_research')
rename_col(stud, 'What kinds of activities did you focus on out of class this week?  [Continuing/Finishing In-Class Work]','finishing_class_work')
rename_col(stud, 'What kinds of activities did you focus on out of class this week?  [Homework Problems/Assignments]','homework')
rename_col(stud, 'What kinds of activities did you focus on out of class this week?  [Out-of-Class Labs]', 'out_of_class_labs')
rename_col(stud, 'What kinds of activities did you focus on out of class this week?  [Multi-Week Project Experience]','multi_week_project')
rename_col(stud, 'What kinds of activities did you focus on out of class this week?  [Other]','other_outside_class')
rename_col(stud, 'What concepts/activities did you or your peers struggle most with this week?','struggle_concepts')
rename_col(stud, 'What questions did you or your peers raise to your instructor/TAs this week?','questions_raised')
rename_col(stud, 'Were there any questions from your peers that surprised you this week?','surprise_questions')

In [0]:
rename_col(profta, 'Timestamp','time')
rename_col(profta,'Participant ID', 'id')
rename_col(profta, 'What is your role in the course you are reflecting on? ','role')
rename_col(profta, 'What topics did you cover this week?','topics_covered')
rename_col(profta, 'What kinds of activities did your students focus  on this week out of class?  [Reading/Research]','reading_research')
rename_col(profta, 'What kinds of activities did your students focus  on this week out of class?  [Continuing/Finishing In-Class Work]','finishing_class_work')
rename_col(profta, 'What kinds of activities did your students focus  on this week out of class?  [Homework Problems/Assignments]','homework')
rename_col(profta, 'What kinds of activities did your students focus  on this week out of class?  [Out-of-Class Labs]', 'out_of_class_labs')
rename_col(profta, 'What kinds of activities did your students focus  on this week out of class?  [Multi-Week Project Experience]','multi_week_project')
rename_col(profta, 'What kinds of activities did your students focus  on this week out of class?  [Other]','other_outside_class')
rename_col(profta, 'By observation, what concepts or processes did students struggle with?','struggle_concepts')
rename_col(profta, 'What questions did students raise this week?','questions_raised')
rename_col(profta, 'Which student questions were surprising to you?','surprise_questions')

# Replace NA values with strings

In [0]:
stud.topics_covered = stud.topics_covered.replace({None:'NA'})
stud.reading_research = stud.reading_research.replace({None:'NA'})
stud.finishing_class_work = stud.finishing_class_work.replace({None:'NA'})
stud.homework = stud.homework.replace({None:'NA'})
stud.out_of_class_labs = stud.out_of_class_labs.replace({None:'NA'})
stud.multi_week_project = stud.multi_week_project.replace({None:'NA'})
stud.other_outside_class = stud.other_outside_class.replace({None:'NA'})
stud.struggle_concepts = stud.struggle_concepts.replace({None:'NA'})
stud.questions_raised = stud.questions_raised.replace({None:'NA'})
stud.surprise_questions = stud.surprise_questions.replace({None:'NA'})

In [0]:
profta.topics_covered = profta.topics_covered.replace({None:'NA'})
profta.reading_research = profta.reading_research.replace({None:'NA'})
profta.finishing_class_work = profta.finishing_class_work.replace({None:'NA'})
profta.homework = profta.homework.replace({None:'NA'})
profta.out_of_class_labs = profta.out_of_class_labs.replace({None:'NA'})
profta.multi_week_project = profta.multi_week_project.replace({None:'NA'})
profta.other_outside_class = profta.other_outside_class.replace({None:'NA'})
profta.struggle_concepts = profta.struggle_concepts.replace({None:'NA'})
profta.questions_raised = profta.questions_raised.replace({None:'NA'})
profta.surprise_questions = profta.surprise_questions.replace({None:'NA'})

# Tokenize function

In [0]:
#sentence tokenizer for bag of words and n-grams, returns tokenized sentences 
#for each column
def tokenize(csv,col):
    whole_text = ''
    for sentence in csv[col]:
        if not str(sentence).endswith(string.punctuation):
            sentence+='. '
            whole_text += str(sentence)
    tokenized = nltk.sent_tokenize(whole_text)
    return tokenized

# Bag of words function

In [0]:
#creates a bag of words for each function
def bag_of_words(tokenized):
    #extends stop words list with context specific frequent words
    stop_words = list(stopwords.words('english'))
    ext = ['students','none','na','think','really','nope','cool','new','vs',
           'particularly','nothing','week','anything','use','learn','heavily',
           'mainly','need','kinds','lots','remember','late','weeks','coming',
           'lot','struggled','using','see','hard','student','cant','different',
           'questions','surprised','pre','processing','n','2','sick']
    stop_words.extend(ext)
    #removes non-alphabetical characters and white spaces
    for i in range(len(tokenized)):
        tokenized[i]=tokenized[i].lower()
        tokenized[i] = re.sub(r'\W',' ',tokenized[i])
        tokenized[i] = re.sub(r'\s+', ' ', tokenized[i])
    #removes blank answers
    for i in reversed(range(len(tokenized))):
        if tokenized[i]==' ':
            del tokenized[i]
    #creates list of lists, inside lists contains sentences tokenized by word
    list_of_lists = []
    for sentence in tokenized:
        tokens = nltk.word_tokenize(sentence)
        sentence_list = []
        #removes stopwords
        for token in tokens:
            if token in stop_words:
                continue
            else:
                sentence_list.append(token)
        list_of_lists.append(sentence_list)
    return list_of_lists

# Create n-grams function

In [0]:
#creates list of n-grams
def extract_ngrams(col,num):
    #creates a list of lists of the sentences tokenized by word
    list_of_lists = []
    #extends stop words with frequent unneccesary words
    stop_words = list(stopwords.words('english'))
    ext = ['class','sick','use','heavily','week','need','questions','see',
           'late','vs','surprised','lot','wayyy','really','lots','hard',
           'student','go','weeks','microsoft']
    stop_words.extend(ext)
    for sentence in col:
      new_sent = ''
      for word in sentence.split():
        #removes stop words
        if word not in stop_words:
          new_sent += word
          new_sent += ' '
      n_grams=ngrams(nltk.word_tokenize(new_sent),num)
      ret = [' '.join(grams) for grams in n_grams]
      list_of_lists.append(ret)
    return list_of_lists

# Create student bags of words


In [11]:
#Topics Covered
stud_topics_tkn = tokenize(stud,'topics_covered')
stud_topics_bag = bag_of_words(stud_topics_tkn)

stud_topics_ngrams = []
ngram_list = extract_ngrams(stud_topics_tkn,2)
for ngram in ngram_list:
    stud_topics_ngrams.append(ngram)
print("Student Topics:",stud_topics_ngrams,'\n')

#Concepts Struggled
stud_struggles_tkn = tokenize(stud,'struggle_concepts')
stud_struggles_bag = bag_of_words(stud_struggles_tkn)

stud_struggles_ngrams = []
ngram_list = extract_ngrams(stud_struggles_tkn,2)
for ngram in ngram_list:
    stud_struggles_ngrams.append(ngram)
print("Student Struggles:",stud_struggles_ngrams,'\n')

#Questions Asked
stud_questions_tkn = tokenize(stud,'questions_raised')
stud_questions_bag = bag_of_words(stud_questions_tkn)

stud_questions_ngrams = []
ngram_list = extract_ngrams(stud_questions_tkn,2)
for ngram in ngram_list:
    stud_questions_ngrams.append(ngram)
print("Student Questions:",stud_questions_ngrams,'\n')

#Surprise Questions
stud_surprise_tkn = tokenize(stud,'surprise_questions')
stud_surprise_bag = bag_of_words(stud_surprise_tkn)

stud_surprise_ngrams = []
ngram_list = extract_ngrams(stud_surprise_tkn,2)
for ngram in ngram_list:
    stud_surprise_ngrams.append(ngram)
print("Student Surprise Questions:",stud_surprise_ngrams,'\n')

Student Topics: [['datetime pandas', 'pandas python'], ['discussed python', 'python functions', 'functions statistics', 'statistics z', 'z score', 'score iqr', 'iqr variability', 'variability standard', 'standard deviation', 'deviation etc', 'etc worked', 'worked projects', 'projects turned', 'turned first', 'first scaffold', 'scaffold activity'], ['python data', 'data types', 'types arrays', 'arrays briefly'], ['arrays tables', 'tables manipulating', 'manipulating rows', 'rows columns', 'columns types', 'types data', 'data presentations'], ['statistics measures', 'measures central', 'central tendency', 'tendency variability', 'variability eda', 'eda terms'], ['sampling distribution', 'distribution random', 'random samples', 'samples population'], ['tables sorting', 'sorting filtering', 'filtering creating', 'creating data', 'data types', 'types access', 'access different', 'different variables', 'variables within', 'within datatype'], ['standard deviation', 'deviation reviewing', 'rev

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: generator 'ngrams' raised StopIteration


# Create Prof./TA Bag of Words

In [12]:
#Topics Covered
profta_topics_tkn = tokenize(profta,'topics_covered')
profta_topics_bag = bag_of_words(profta_topics_tkn)

profta_topics_ngrams = []
ngram_list = extract_ngrams(profta_topics_tkn,2)
for ngram in ngram_list:
    profta_topics_ngrams.append(ngram)
print("Professor/TA Topics:",profta_topics_ngrams,'\n')

#Concepts Struggled
profta_struggles_tkn = tokenize(profta,'struggle_concepts')
profta_struggles_bag = bag_of_words(profta_struggles_tkn)

profta_struggles_ngrams = []
ngram_list = extract_ngrams(profta_struggles_tkn,2)
for ngram in ngram_list:
    profta_struggles_ngrams.append(ngram)
print("Professor/TA Struggles:",profta_struggles_ngrams,'\n')

#Questions Asked
profta_questions_tkn = tokenize(profta,'questions_raised')
profta_questions_bag = bag_of_words(profta_questions_tkn)

profta_questions_ngrams = []
ngram_list = extract_ngrams(profta_questions_tkn,2)
for ngram in ngram_list:
    profta_questions_ngrams.append(ngram)
print("Professor/TA Questions:",profta_questions_ngrams,'\n')

#Surprise Questions
profta_surprise_tkn = tokenize(profta,'surprise_questions')
profta_surprise_bag = bag_of_words(profta_surprise_tkn)

profta_surprise_ngrams = []
ngram_list = extract_ngrams(profta_surprise_tkn,2)
for ngram in ngram_list:
    profta_surprise_ngrams.append(ngram)
print("Professor/TA Surprise Questions:",profta_surprise_ngrams,'\n')

Professor/TA Topics: [['meeting scheduling', 'scheduling project', 'project group', 'group introductions', 'introductions organization'], ['github idear', 'idear exploratory', 'exploratory data', 'data analysis', 'analysis data', 'data acquisition'], ['data acquisition', 'acquisition exploration', 'exploration github', 'github setups'], ['data acquisition', 'acquisition pre', 'pre processing'], ['barplots color', 'color theory', 'theory data', 'data collection'], ['midterm review', 'review data', 'data viz', 'viz topics', 'topics trend', 'trend lines', 'lines regression', 'regression loess', 'loess shiny', 'shiny interactive', 'interactive visualizations', 'visualizations rstudio', 'rstudio project'], ['intro data', 'data wrangling', 'wrangling working', 'working projects'], ['data wrangling', 'wrangling grouping', 'grouping rows', 'rows summarizing', 'summarizing rows', 'rows creating', 'creating new', 'new variables', 'variables sorting', 'sorting rows', 'rows dataset'], ['joining me

# Fitting TFIDF Model 

In [0]:
def tfidfModelling(bag):
  dictionary = Dictionary(bag)
  corpus = [dictionary.doc2bow(doc) for doc in bag]
  return(TfidfModel(corpus),corpus,dictionary)

#Top 10 tf-idf words 



In [0]:
def top10Idf (bag):
  model, corpus,dictionary = tfidfModelling(bag)
  weights = []
  for doc in corpus:
    for combo in model[doc]:
      weights.append(combo)
  #sorted weighted list
  weights = sorted(weights, key=lambda w : w[1], reverse=True)
  new_weights = []
  #make list unique
  for word in weights:
    if word not in new_weights:
      new_weights.append(word)
  #print top 10 words based on tfidf
  for term_id, weight in new_weights[0:10]:
    print(dictionary.get(term_id), weight)
  print("\n") 
  #return top 10 words based on tfidf
  return [(dictionary[pair[0]],pair[1]) for pair in new_weights[0:10]]


# Prints the Top 10 for Prof/TA & Students

In [15]:
#Topics Covered
print("Professor/TA Topics:")
top_profta_topics = top10Idf(profta_topics_bag)
print("Student Topics:")
top_stud_topics = top10Idf(stud_topics_bag)

#Struggles Encountered
print("Professor/TA Struggles:")
top_profta_struggles = top10Idf(profta_struggles_bag)
print("Student Struggles:")
top_stud_struggles = top10Idf(stud_struggles_bag)

#Questions Asked
print("Professor/TA Questions:")
top_profta_questions = top10Idf(profta_questions_bag)
print("Student Questions:")
top_stud_questions = top10Idf(stud_questions_bag)

#Surprise Questions Asked
print("Professor/TA Surprise Questions:")
top_profta_surprise = top10Idf(profta_surprise_bag)
print("Student Surprise Questions:")
top_stud_surprise = top10Idf(stud_surprise_bag)

Professor/TA Topics:
github 1.0
data 1.0
idf 1.0
preprocessing 0.9679195717383295
acquisition 0.9573122438649556
iteration 0.9326446771245245
worked 0.852508574458651
rows 0.7613948638501783
types 0.7311619496050147
learning 0.7001663757575962


Student Topics:
tidy 1.0
types 0.966895707705219
wrangling 0.9210412666697719
projects 0.752896516222731
tree 0.7363185426507128
language 0.7071067811865475
natural 0.7071067811865475
bayes 0.7071067811865475
naive 0.7071067811865475
clustering 0.6666666666666667


Professor/TA Struggles:
shiny 1.0
summarize 1.0
programming 1.0
wrangling 0.8851521213595946
terminology 0.8809504870645746
grasping 0.8
commandline 0.7071067811865476
navigating 0.7071067811865476
dataset 0.7071067811865476
structuring 0.7071067811865476


Student Struggles:
class 1.0
tidy 0.9087520211324654
summary 0.7664881671343302
distribution 0.7353011379117274
deviation 0.7071067811865476
standard 0.7071067811865476
material 0.7071067811865476
midterm 0.7071067811865476
exam 0

# Prints the top 10 N-Grams 

In [16]:
#Topics Covered
print("Professor/TA Topics:")
top_profta_topics2 = top10Idf(profta_topics_ngrams)
print("Student Topics:")
top_stud_topics2 = top10Idf(stud_topics_ngrams)

#Struggles Encountered
print("Professor/TA Struggles:")
top_profta_struggles2 = top10Idf(profta_struggles_ngrams)
print("Student Struggles:")
top_stud_struggles2 = top10Idf(stud_struggles_ngrams)

#Questions Asked
print("Professor/TA Questions:")
top_profta_questions2 = top10Idf(profta_questions_ngrams)
print("Student Questions:")
top_stud_questions2 = top10Idf(stud_questions_ngrams)

#Surprise Questions Asked
print("Professor/TA Surprise Questions:")
top_profta_surprise2 = top10Idf(profta_surprise_ngrams)
print("Student Surprise Questions:")
top_stud_surprise2 = top10Idf(stud_surprise_ngrams)

Professor/TA Topics:
iteration github 1.0
data preprocessing 1.0
processing data 0.7783666410061783
meetings client 0.7783666410061783
students worked 0.7071067811865476
worked projects 0.7071067811865476
maps practice 0.7071067811865476
practice exam 0.7071067811865476
gis api 0.7071067811865476
maps gis 0.7071067811865476


Student Topics:
data wrangling 1.0
data types 1.0
naive bayes 1.0
students worked 0.7694780625232275
datetime pandas 0.7071067811865475
pandas python 0.7071067811865475
maps practice 0.7071067811865475
practice exam 0.7071067811865475
language processing 0.7071067811865475
natural language 0.7071067811865475


Professor/TA Struggles:
grasping github 1.0
early introduce 1.0
data wrangling 1.0
github terminology 1.0
navigating commandline 1.0
dataset structuring 1.0
naive bayes 1.0
grasping naive 0.7729084932359204
management collaboration 0.7729084932359204
presentation skills 0.7071067811865476


Student Struggles:
standard deviation 1.0
tidy data 1.0
summary stat

# Function to make Word Cloud

In [0]:
def wordcloud(dictionary):
  wc = WordCloud(background_color='black',max_words=2000,width=1024,height=720,colormap="spring")
  wc.generate_from_frequencies(dict(dictionary))
  return wc

# Function to Plot Word Clouds in a Grid

In [0]:
def plotclouds(wc1,wc2,wc3,wc4,save):
  plt.subplot(2,2,1).imshow(wc1,interpolation='bilinear')
  plt.title("Tfidf")
  plt.axis("off")
  plt.subplot(2,2,2).imshow(wc2,interpolation='bilinear')
  plt.title("NGrams Tfidf")
  plt.axis("off")
  plt.subplot(2,2,3).imshow(wc3,interpolation='bilinear')
  plt.axis("off")
  plt.subplot(2,2,4).imshow(wc4,interpolation='bilinear')
  plt.axis("off")
  plt.savefig(save)
  plt.clf()

# Plotting Word Clouds

In [19]:
#Topics Covered
wc1 = wordcloud(top_profta_topics)
wc2 = wordcloud(top_profta_topics2)
wc3 = wordcloud(top_stud_topics)
wc4 = wordcloud(top_stud_topics2)
plotclouds(wc1,wc2,wc3,wc4,'cloud_topics')

<Figure size 432x288 with 0 Axes>

In [20]:
#Questions Asked
wc1 = wordcloud(top_profta_questions)
wc2 = wordcloud(top_profta_questions2)
wc3 = wordcloud(top_stud_questions)
wc4 = wordcloud(top_stud_questions2)
plotclouds(wc1,wc2,wc3,wc4,'cloud_questions')

<Figure size 432x288 with 0 Axes>

In [21]:
#Struggles Encountered
wc1 = wordcloud(top_profta_struggles)
wc2 = wordcloud(top_profta_struggles2)
wc3 = wordcloud(top_stud_struggles)
wc4 = wordcloud(top_stud_struggles2)
plotclouds(wc1,wc2,wc3,wc4,'cloud_struggles')

<Figure size 432x288 with 0 Axes>

In [22]:
#Surprise Questions Asked
wc1 = wordcloud(top_profta_surprise)
wc2 = wordcloud(top_profta_surprise2)
wc3 = wordcloud(top_stud_surprise)
wc4 = wordcloud(top_stud_surprise2)
plotclouds(wc1,wc2,wc3,wc4,'cloud_surprise')

<Figure size 432x288 with 0 Axes>